## This is the initial network design for chromstem-net

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

This is the dataset generator class, to be loaded into the DataLoader tool

This is adapted from the pytorch tutorial: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [88]:
class ChromstemDataset(Dataset):
    """Chromstem Dataset"""
    
    def __init__(self,csv_file,root_dir,transform=None):
        """
        Args:
            csv_file (string): Master CSV file that will hold all the data
            root_dir (string): Directory where the datasets are labeled
            transform (callable, optional): Optional transformations to samples
        """
        
        f = open(csv_file,'r')
        csv_length = len(f.readlines()[-1].split(','))
        f.close()
        col_names = ['File'] + ['Nucl%d' % i for i in range(csv_length-1)]
        self.nucl_coords_frame_ = pd.read_csv(csv_file,engine='python',header=None,names=col_names)
        self.root_dir_ = root_dir
        self.transform_ = transform
    
    def read_chromstem(self,fnme):
        x,y,z,rho = np.loadtxt(fnme,comments='#',unpack=True)
        tensor = torch.zeros_like(torch.empty(1,int(x[0]),int(y[0]),int(z[0])))
    
        for i,arr in enumerate(zip(x[1:],y[1:],z[1:])):
            tensor[0,int(arr[0]),int(arr[1]),int(arr[2])] = rho[i]
        
        # Convert to sparse matrix as data will be sparse
        #tensor = torch.Sparse.FloatTensor(tensor)
        return tensor
    
    def __len__(self):
        return(len(self.nucl_coords_frame_))
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        chromstem_name = os.path.join(self.root_dir_,self.nucl_coords_frame_.iloc[idx,0])
        chromstem = self.read_chromstem(chromstem_name)
        nucl_coords = self.nucl_coords_frame_.iloc[idx,1:]
        nucl_coords = np.asarray([nucl_coords])
        nucl_coords = nucl_coords.astype('float').reshape(-1,3)
        print(chromstem_name)
        num_nucls = chromstem_name.strip('/data/')[1]
        
        sample = {'chromstem' : chromstem, 'nucl_coords' : nucl_coords, 'num_nucls' : num_nucls}
        
        if self.transform_:
            sample = self.transform_(sample)
            
        return sample
        

Here, would be useful to provide a plotting function for samples in the dataset.
The main input to this function is any sample from the dataset and it would output the density and "centromere" in a 3D plot.
To do so accurately, I am going to use the ipyvolume widget.

In [4]:
import numpy as np
import ipyvolume as ipv

In [5]:
def plot_sample(sample):
    ipv.figure()
    # Plot voxels of density
    ipv.scatter(sample['chromstem'][:,0],sample['chromstem'][:,1],sample['chromstem'][:,2], 
                color='blue',size=10,marker='box',opacity=0.5)
    
    # Plot nucleosome centers
    ipv.scatter(sample['nucl_coords'][:,0],sample['nucl_coords'][:,1],sample['nucl_coords'][:,2],
               color='red',size=5,marker='sphere')
    
    ipv.show()
    return

This is a quick test of the widget and plotting function before the data is loaded

In [6]:
sample = {'chromstem' :  np.asarray([[1,2,3]]), 'nucl_coords' : np.asarray([[0,1,2],[5,10,0.2],[10,3.3,2]])}
plot_sample(sample)

Here is where the dataset is loaded in. The .csv file should be located in the current directory, so the root is '.'

In [89]:
chromstem_dataset = ChromstemDataset(csv_file='output_label_test.csv',
                                     root_dir='.')

Here is a quick test to make sure the dataset looks appropriate

In [90]:
print(chromstem_dataset[20])

./data/5/0/data-117.dat
{'chromstem': tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 2.8299, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
    

Now I will load in the entire dataset and begin to feed it into a built neural network

In [30]:
chromstem_dataset = ChromstemDataset(csv_file='output_label.csv',
                                     root_dir='.')

To start, the dataloader will be able to shuffle the data, and I will start with a batch size of 4. No idea if that will work, but for now this is a fixed value. 

In [73]:
trainloader = DataLoader(chromstem_dataset, batch_size=1,
                        shuffle=True, num_workers=1)

testloader  = DataLoader(chromstem_dataset, batch_size=1,
                         shuffle=False, num_workers=1)

Finally, the neural network class will be initialized here.

In [74]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv3d(1,32,3)
        self.pool = nn.MaxPool3d(2,2)
        self.conv2 = nn.Conv3d(32,64,3)
        self.conv3 = nn.Conv3d(64,128,3)
        self.fc1   = nn.Linear(512,512)
        self.fc2   = nn.Linear(512,512)
        self.fc3   = nn.Linear(512,3)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 512)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

To start, the cross entropy loss function is chosen, however a different function may be required

In [24]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

If the user has CUDA functionality on their computer, the network trains significantly faster with GPU. As such, here I will provide the opportunity to use CUDA. A correct output would say that the device being used is "cuda:0"

In [75]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

net.to(device)

cuda:0


Net(
  (conv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=3, bias=True)
)

Train the network briefly here

In [78]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['chromstem'].to(device), data['nucl_coords'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

tensor([[ 0.0123,  0.0070, -0.0263],
        [ 0.0040,  0.0055, -0.0219]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[[-307.7910, -224.1900,  475.1330],
         [-117.0700, -430.4780,  460.0250],
         [ -61.9396, -479.7300,  451.7170],
         [ 115.7660,  429.6730, -435.6300],
         [-325.5500, -214.1740, -451.6170],
         [-253.3640, -191.4080, -444.1780],
         [  -8.4756, -182.8690,  409.1870],
         [ 328.7910, -204.9650, -410.0590],
         [ 395.8940,  -50.9949, -460.2540],
         [ -98.1361,  -33.9312, -421.9930],
         [ -33.4192,  -49.0743, -459.6620],
         [ 358.7770,  -45.0992,  470.0540],
         [ 391.4000,  250.9840,  497.4440],
         [-308.8720,  153.8500, -331.3600],
         [ -65.6043,  329.3810, -456.6500],
         [ 201.6960,  107.7010, -330.9460],
         [ 374.5780,  477.9260, -185.1700],
         [-249.1820, -485.9140,  -77.6638],
         [ -83.9149, -358.3000, -193.3860],
         [ 358.9040,  451.4890, -272.5600],
   

ValueError: Expected input batch_size (2) to match target batch_size (1).